In [ ]:
import os, getpass

def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")

In [ ]:
from langchain_openai import ChatOpenAI, OpenAI

def multiply(a: int, b: int) -> int:
    """Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a * b

llm = ChatOpenAI(model="gpt-4o")
llm_with_tools = llm.bind_tools([multiply])


In [ ]:
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display



def tool_calling_llm(state: MessagesState):
    return {"messages": [llm_with_tools.invoke(state['messages'])]}

builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode([multiply]))
builder.add_edge(START, "tool_calling_llm")
builder.add_conditional_edges(
    "tool_calling_llm",
    # If the latest message result from the llm is a tool call, tool_condition routes to tools
    # If not tool call, tool_condition routes to END
    tools_condition
)
builder.add_edge("tools", END) # since it is the only route without END
graph = builder.compile()

display(Image(graph.get_graph().draw_mermaid_png()))


In [12]:
from langchain_core.messages import HumanMessage
messages = [HumanMessage(content="Hello, what is 2 multipled by 2?")]
messages = graph.invoke({"messages": messages})
for m in messages['messages']:
    m.pretty_print()

================================ Human Message =================================

Hello, what is 2 multipled by 2?
================================== Ai Message ==================================
Tool Calls:
  multiply (call_tVkhwQ5oWMT9aorj4n71U2SI)
 Call ID: call_tVkhwQ5oWMT9aorj4n71U2SI
  Args:
    a: 2
    b: 2
================================= Tool Message =================================
Name: multiply

4
